<a href="https://colab.research.google.com/github/bmurphy2019/groupProjectAi/blob/main/diabetesAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Brian Murphy R00189335
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint

# import the file
df = pd.read_csv('/content/diabetes.csv',)

df.duplicated().sum()
df.drop_duplicates(inplace=True)

columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
# data cleaning
for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
 
df.dropna(inplace=True)

#assign target to Y everything else to X
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# 80-20 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

# define the model
model = Sequential()
model.add(Dense(8, activation = 'relu', input_shape = X_train.iloc[0].shape))

model.add(Dense(8, activation='relu'))

model.add(Dense(4, activation='relu')) 

model.add(Dense(1, activation='sigmoid')) 

model.compile(optimizer= 'adam' ,loss='binary_crossentropy',metrics=['acc'])



# compile and checkpoint
checkpointer = ModelCheckpoint('diabetes.h5', monitor='val_acc', mode='max', verbose=2, save_best_only=True)
history=model.fit(X_train, y_train, batch_size=16, epochs=350, validation_data=(X_test, y_test), callbacks=[checkpointer])

Epoch 1/350
 1/19 [>.............................] - ETA: 15s - loss: 6.9498 - acc: 0.4375
Epoch 00001: val_acc improved from -inf to 0.25974, saving model to diabetes.h5
19/19 [==============================] - 1s 27ms/step - loss: 6.2633 - acc: 0.3355 - val_loss: 4.8216 - val_acc: 0.2597
Epoch 2/350
 1/19 [>.............................] - ETA: 0s - loss: 4.8215 - acc: 0.3125
Epoch 00002: val_acc improved from 0.25974 to 0.35065, saving model to diabetes.h5
19/19 [==============================] - 0s 8ms/step - loss: 2.4987 - acc: 0.3454 - val_loss: 1.1776 - val_acc: 0.3506
Epoch 3/350
19/19 [==============================] - ETA: 0s - loss: 0.8507 - acc: 0.5395
Epoch 00003: val_acc improved from 0.35065 to 0.64935, saving model to diabetes.h5
19/19 [==============================] - 0s 9ms/step - loss: 0.8507 - acc: 0.5395 - val_loss: 0.7181 - val_acc: 0.6494
Epoch 4/350
 1/19 [>.............................] - ETA: 0s - loss: 0.9060 - acc: 0.5000
Epoch 00004: val_acc did not improv

In [2]:
# simple evaluate and print accuracy
present_model = keras.models.load_model('diabetes.h5')
print("Accuracy of our model on test data : " , present_model.evaluate(X_test,y_test)[1]*100 , "%")

3/3 [==============================] - 0s 4ms/step - loss: 0.4392 - acc: 0.8442
Accuracy of our model on test data :  84.41558480262756 %


In [3]:
# convert and output file to tflite model
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(keras.models.load_model('diabetes.h5'))
tflite_model = converter.convert()

# Save the model.
with open('diabetes.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp9nglpdj0/assets
